In [1]:
import numpy as np
import pandas as pd
import math
import sys
from ipynb.fs.full.Formulas import *
from ipynb.fs.full.Environment import *
from ipynb.fs.full.Flow_Grouping_Algo import *
import random

Bhaichara On Top


In [3]:
def concurrent_transmission():
    groups = generate_groups()
    system_throughput = 0
    hops = np.ones(number_of_flows + 1)
    
    for current_time_slot in range(1, number_of_time_slots + 1):
        for group in groups:
            while(len(group)):
                